In [ ]:
import fastai
from fastai import *
from fastai.vision import *
import torch
print(torch.__version__)
print(fastai.__version__)

## 4.3 fastai


如果你已经学会用fastai创建实用的计算机视觉（CV）模型，那你就可以用同样的方法创建自然语言处理（NLP）模型，或是其他模型。


### fastai实践
#### MNIST
我们还是以最简单的MNIST来入手看看fastai都为我们做了什么



In [ ]:
path = untar_data(URLs.MNIST_SAMPLE)


URLs.MNIST_SAMPLE 只提供了3和7 两个分类的数据，这个是用来做演示的

这里如果下载很慢的话，那么我们可以手动进行操作（建议这样，比程序下载快很多而且稳定）

```
#进入我们用户目录，创建以下的目录
mkdir -p ~/.fastai/data
cd ~/.fastai/data
# 下载解压
wget -c http://files.fast.ai/data/examples/mnist_sample.tgz
tar -zxvf mnist_sample.tgz
```

完成后重新执行上面的命令即可

In [ ]:
#使用ImageDataBunch从刚才的目录中将读入数据
data = ImageDataBunch.from_folder(path)

In [ ]:
data

In [ ]:
# 使用cnn_learner来创建一个learn，这里模型我们选择resnet18，使用的计量方法是accuracy准确率
learn = creat_cnn(data, models.resnet18, metrics = accuracy)

In [ ]:
# 可以直接使用train_ds来访问数据集里面的数据
img, label = data.train_ds[0]
print(label)
img

In [ ]:
#或者我们直接使用show_batch方法，连标签都给我们自动生成好了
data.show_batch(rows = 3, figsize = (6, 6))

这里也是直接下载PyTorch官方提供的resnet18与训练模型

```
wget -P /Users/tant/.torch/models/ https://download.pytorch.org/models/resnet18-5c106cde.pth

```

In [ ]:
# 使用learn的fit方法就可以进行训练了，训练一遍
learn.fit()

下面再介绍一些训练的高级用法

In [ ]:
# 重新生成一个数据集
learn2 = creat_cnn(data, models,resnet18, metrics = accuracy, callback_fns = ShowGraph)

这里我们使用`fit_one_cycle`方法。

`fit_one_cycle`使用的是一种周期性学习率，从较小的学习率开始学习，缓慢提高至较高的学习率，然后再慢慢下降，周而复始，每个周期的长度略微缩短，在训练的最后部分，允许学习率比之前的最小值降得更低。这不仅可以加速训练，还有助于防止模型落入损失平面的陡峭区域，使模型更倾向于寻找更平坦的极小值，从而缓解过拟合现象。

In [ ]:
learn2.fit_one_cycle(1)

我们使用内置ShowGraph的方法直接打印训练的状态，如果我们需要更详细的状态，可以直接调用一下的方法：

In [ ]:
# 学习率的变更
learn2.recorder.plot_lr()

In [ ]:
#损失
learn2.recorder.plot_losses()

In [ ]:
# 我们也可以使用lr_find()找到损失仍在明显改善最高学习率
learn2.lr_find()
learn2.recorder.plot()